In [1]:
!pip -q install ipywidgets
!pip -q install google-genai
!pip -q install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

from IPython.display import HTML, Markdown

# Instalar Framework ADK de agentes do Google ################################################

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [3]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [4]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [10]:
##########################################
# --- Agente 1: Descritor --- #
##########################################
def agente_descritor(personagem, estilo, tematica, referencia, cor, habilidade, vestimenta):

    descritor = Agent(
        name="agente_descritor",
        model="gemini-2.0-flash",
        description="Agente que descreve com mais detalhes usando as informações base",
        tools=[google_search],
        instruction="""
          Você é um game designer. A sua tarefa é usar a ferramenta de busca do google (google_search)
          para buscar referencias do personagem descrito e expandir conceitos, seu foco é manter a essencia
          e ao mesmo tempo detalhar mais os valores destacados no prompt
        """
    )

    entrada_do_agente_descritor = f"""
    Personagem: {personagem}\n
    Estilo: {estilo}\n
    Temática: {tematica}\n
    Referência: {referencia}\n
    Cor: {cor}\n
    Habilidade: {habilidade}\n
    Vestimenta: {vestimenta}\n
    """

    descricao = call_agent(descritor, entrada_do_agente_descritor)
    return descricao

In [11]:
######################################
# --- Agente 3: Gerador --- #
######################################
def agente_gerador(personagem, estilo, tematica, referencia, cor, habilidade, vestimenta, personagem_refinado):
    gerador = Agent(
        name="agente_gerador",
        model="gemini-2.0-flash",
        instruction="""
            Você é um gerador de prompt de Imagens para modelos de IA. A sua tarefa é receber uma descrição para a geração desse personagem
            A imagem será gerada em uma resolução de 378x378 sendo 3x3 de 128 totalizando 9 frames de animação
            A imagem será gerado em um formato de spritesheet com 9 frames de animação estando o personagem centralizado no meio,
            A imagem ter consistencia é o fator mais importante na geração desse personagem elementos não podem surgir ou desaparecer,
            A imagem deve ler a imagem anterior como parâmetro para gerar a imagem seguinte,
            A imagem é essencial manter a essencia dos detalhes providos e considerar a descrição aprofundada
            e gerar um prompt detalhado para a geração da imagem
            """,
        description="Agente gerador de imagens"
    )
    entrada_do_agente_gerador = f"""
    Personagem: {personagem}\n
    Estilo: {estilo}\n
    Temática: {tematica}\n
    Referência: {referencia}\n
    Cor: {cor}\n
    Habilidade: {habilidade}\n
    Vestimenta: {vestimenta}\n
    #####
    Descrição: {personagem_refinado}
    """
    # Executa o agente
    prompt_imagem = call_agent(gerador, entrada_do_agente_gerador)
    return prompt_imagem

In [12]:
##########################################
# --- Agente 4: Ajustador --- #
##########################################
def agente_revisor(personagem, estilo, tematica, referencia, cor, habilidade, vestimenta, prompt_imagem):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um revisador de prompt de imagens. Seu papel é fazer o ajuste final para garantir que o prompt de imagem gere um resultado consistente
            Garanta que a imagem será gerada em uma resolução de 378x378 sendo 3x3 de 128 totalizando 9 frames de animação
            Garanta que a imagem será gerado em um formato de spritesheet com 9 frames de animação estando o personagem centralizado no meio
            """,
        description="Agente revisor de detalhes de imagem."
    )
    entrada_do_agente_revisor = f"""
    Personagem: {personagem}\n
    Estilo: {estilo}\n
    Temática: {tematica}\n
    Referência: {referencia}\n
    Cor: {cor}\n
    Habilidade: {habilidade}\n
    Vestimenta: {vestimenta}\n
    #####
    Prompt de Imagem: {prompt_imagem}
    """
    # Executa o agente
    prompt_final = call_agent(revisor, entrada_do_agente_revisor)
    return prompt_final

In [13]:
import ipywidgets as widgets
from IPython.display import display

# Cria os widgets do formulário (sem estilos)
personagem = widgets.Text(description="Personagem:")
estilo = widgets.Dropdown(description="Estilo:", options=['8-bit', '16-bit', '32-bit', '64-bit'])
tematica = widgets.Dropdown(description="Temática:", options=['RPG', 'Fantasia', 'Aventura', 'Ação', 'Ficção Científica', 'Terror/Horror', 'Steampunk', 'Cyberpunk', 'Mitologia', 'Histórico', 'Cartoon', 'Educacional'])
referencia = widgets.Text(description="Referência:")
cor = widgets.Text(description="Cor Principal:")
habilidade = widgets.Text(description="Habilidade:")
vestimenta = widgets.Text(description="Vestimenta:")
botao_enviar = widgets.Button(description="Enviar")
output = widgets.Output()

# Aplica estilos diretamente aos widgets (removendo qualquer personalização)
personagem.style.description_width = ''
estilo.style.description_width = ''
tematica.style.description_width = ''
referencia.style.description_width = ''
cor.style.description_width = ''
habilidade.style.description_width = ''
vestimenta.style.description_width = ''
botao_enviar.style.button_color = None
botao_enviar.style.text_color = None

# Define a função para lidar com o clique do botão
def ao_clicar_botao(b):
    with output:
        output.clear_output()
        print(f"Personagem: {personagem.value}")
        print(f"Estilo artístico: {estilo.value}")
        print(f"Temática: {tematica.value}")
        print(f"Referencia: {referencia.value}")
        print(f"Cor: {cor.value}")
        print(f"Habilidade: {habilidade.value}")
        print(f"Vestimenta: {vestimenta.value}")

        descricao = agente_descritor(personagem.value, estilo.value, tematica.value, referencia.value, cor.value, habilidade.value, vestimenta.value)
        print("\n--- Resultado do Agente 1 (Descritor) ---\n")
        display(to_markdown(descricao))
        print("\n----------------------------------------\n")

        prompt_imagem = agente_gerador(personagem.value, estilo.value, tematica.value, referencia.value, cor.value, habilidade.value, vestimenta.value, descricao)
        print("\n--- Resultado do Agente 3 (Gerador) ---\n")
        display(to_markdown(prompt_imagem))
        print("\n----------------------------------------\n")

        post_final = agente_revisor(personagem.value, estilo.value, tematica.value, referencia.value, cor.value, habilidade.value, vestimenta.value, prompt_imagem)
        print("\n--- Resultado do Agente 4 (Ajustador) ---\n")
        display(to_markdown(post_final))
        print("\n----------------------------------------\n")

# Associa a função ao evento de clique do botão
botao_enviar.on_click(ao_clicar_botao)

# Organiza os widgets em um layout vertical
formulario = widgets.VBox([personagem, estilo, tematica, referencia, cor, habilidade, vestimenta, botao_enviar, output])

# Exibe o formulário
display(formulario)